# Phase 5 Orchestrator Demo
This notebook replays a synthetic orchestration scenario using the simulation harness introduced in Phase 5.
It showcases how negotiation outcomes, guardrail decisions, and success metrics are captured for developer analysis.

In [ ]:
import asyncio
from pprint import pprint

from app.agents.base import AgentContext
from app.monitoring.orchestrator_report import build_evaluation_snapshot
from app.orchestration.simulation import SimulationHarness, SimulationScenario

class DemoOrchestrator:
    async def route_task(self, task: dict, *, context: AgentContext, progress_cb=None) -> dict:
        if progress_cb is not None:
            await progress_cb({"event": "started", "task_id": task.get("id")})
        await asyncio.sleep(0.05)
        outputs = [
            {
                "agent": "researcher",
                "summary": f"Completed: {task['prompt']}",
                "confidence": 0.82,
                "metadata": {"rationale": "Validated against knowledge base"},
            }
        ]
        return {
            **task,
            "status": "completed",
            "outputs": outputs,
            "negotiation": {"status": "completed", "consensus": 0.75, "metadata": {"strategy": "demo"}},
            "guardrails": {"decisions": []},
            "plan": {"status": "planned"},
        }

def context_factory(task: dict) -> AgentContext:
    return AgentContext(memory=None, llm=None)

orchestrator = DemoOrchestrator()
harness = SimulationHarness(orchestrator, context_factory=context_factory)
scenario = SimulationScenario(
    name="demo-negotiation",
    base_task={"prompt": "Draft pilot launch brief"},
    repetitions=3,
    concurrency=2,
)

report = await harness.run(scenario)
snapshot = build_evaluation_snapshot(report)
print(f"Success rate: {report.success_rate:.2f}")
print(f"Average latency: {report.average_latency:.3f}s")
print(f"Escalation rate: {report.escalation_rate:.2f}")
pprint(snapshot)

In [ ]:
for idx, run in enumerate(report.runs, start=1):
    print(f"Run {idx}: status={run.status}, latency={run.latency_seconds:.3f}s, consensus={run.negotiation_consensus}")